# Reporting on TPC QC for LHC22c [apass3] (and comparison to apass2)

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#apass3-overview" data-toc-modified-id="apass3-overview-1">apass3 overview</a></span><ul class="toc-item"><li><span><a href="#Clusters" data-toc-modified-id="Clusters-1.1">Clusters</a></span></li><li><span><a href="#Tracks-observables" data-toc-modified-id="Tracks-observables-1.2">Tracks observables</a></span></li></ul></li><li><span><a href="#Comparing-apass3-to-apass2-with-trends" data-toc-modified-id="Comparing-apass3-to-apass2-with-trends-2">Comparing apass3 to apass2 with trends</a></span><ul class="toc-item"><li><span><a href="#pT-trending" data-toc-modified-id="pT-trending-2.1">pT trending</a></span></li><li><span><a href="#Time-gain-calibration" data-toc-modified-id="Time-gain-calibration-2.2">Time gain calibration</a></span></li></ul></li><li><span><a href="#apass3-only-trends-with-checks" data-toc-modified-id="apass3-only-trends-with-checks-3">apass3 only trends with checks</a></span></li><li><span><a href="#Plots-from-apass3" data-toc-modified-id="Plots-from-apass3-4">Plots from apass3</a></span></li><li><span><a href="#Clusters-QC" data-toc-modified-id="Clusters-QC-5">Clusters QC</a></span></li><li><span><a href="#Tracks-QC" data-toc-modified-id="Tracks-QC-6">Tracks QC</a></span><ul class="toc-item"><li><span><a href="#Geometrical-distributions-of-tracks" data-toc-modified-id="Geometrical-distributions-of-tracks-6.1">Geometrical distributions of tracks</a></span></li><li><span><a href="#Eta-distribution" data-toc-modified-id="Eta-distribution-6.2">Eta distribution</a></span></li><li><span><a href="#Track-properties" data-toc-modified-id="Track-properties-6.3">Track properties</a></span></li><li><span><a href="#Number-of-clusters-per-track" data-toc-modified-id="Number-of-clusters-per-track-6.4">Number of clusters per track</a></span></li><li><span><a href="#nClusters-vs-phi" data-toc-modified-id="nClusters-vs-phi-6.5">nClusters vs phi</a></span></li><li><span><a href="#nClusters-vs-eta" data-toc-modified-id="nClusters-vs-eta-6.6">nClusters vs eta</a></span></li><li><span><a href="#nClusters-vs-transverse-momentum" data-toc-modified-id="nClusters-vs-transverse-momentum-6.7">nClusters vs transverse momentum</a></span></li><li><span><a href="#PID" data-toc-modified-id="PID-6.8">PID</a></span></li></ul></li><li><span><a href="#TPC-Gain-calibration" data-toc-modified-id="TPC-Gain-calibration-7">TPC Gain calibration</a></span><ul class="toc-item"><li><span><a href="#dEdx-vs-p" data-toc-modified-id="dEdx-vs-p-7.1">dEdx vs p</a></span></li></ul></li></ul></div>

In [ ]:
%%capture
import ROOT
import glob
import math
import numpy as np
import pandas as pd
from IPython.display import display, Markdown, HTML
import ipywidgets as widgets
from TPCQCVis.src.drawHistograms import *
from TPCQCVis.src.drawTrending import *
from TPCQCVis.src.drawMultiTrending import *
from TPCQCVis.src.checkHistograms import *
from TPCQCVis.src.checkTrending import *

In [ ]:
display(HTML("<style>.container { width:95% !important; }</style>"))
display(HTML("<style>table {float:left;}</style>"))

In [ ]:
# Read the Root Files
path = "/cave/alice/data/2022/LHC22c/apass3/"
passName = "apass3"
fileList = glob.glob(path+"*_QC.root")
fileList.sort()
#fileList = fileList[13:]
runList = [fileList[i][36:-8] for i in range(len(fileList))]
rootDataFile=[]
for file in fileList:
    rootDataFile.append(ROOT.TFile.Open(file,"READ"))
#len(fileList)

In [ ]:
# Read the Root Files
path = "/cave/alice/data/2022/LHC22c/apass2/"
pass1Name = "apass2"
fileList_apass1  = []
for x in fileList:
    line = list(x)
    line[-16] = "2"
    fileList_apass1.append("".join(line))

fileList_apass1 = glob.glob(path+"*_QC.root")
fileList_apass1.sort()
runList_apass1 = [fileList_apass1[i][-14:-8] for i in range(len(fileList_apass1))]
rootDataFile_apass1=[]
for file in fileList_apass1:
    rootDataFile_apass1.append(ROOT.TFile.Open(file,"READ"))

---
## apass3 overview

### Clusters

In [ ]:
qualityDFClusters = pd.DataFrame({'runNumber':runList})
names = []
for i in range(72):
    if i < 18*2: roc = "IROC-"
    else: roc = "OROC-"
    if i%(18*2) < 18: side = "A"
    else : side = "C"
    names.append(roc+side+str(i%18).zfill(2))

In [ ]:
objectName="c_ROCs_N_Clusters_1D;1"
trending = "mean"
error = ""
[trends,canvas] = drawMultiTrending(objectName,rootDataFile,names=runList,namesFromRunList=True,
                               trend=trending,error=error,pads=True,normalize=True,
                               log="none",yAxisRange=[0,500])
for iPad,pad in enumerate(canvas.GetListOfPrimitives()):
    trend = trends[iPad]
    [qualities,pad] = checkTrending(trend,canvas=pad,thresholds={"GOOD":3,"MEDIUM":6,"BAD":9})
    qualityDFClusters[names[iPad]] = qualities
    canvas.Update()

In [ ]:
dfPrintClusters = pd.DataFrame(columns=["runNumber","problematicROCs"])
for index, row in qualityDFClusters.iterrows():
    runNumber = row["runNumber"]
    problematic = []
    for column in qualityDFClusters.columns:
        if row[column] == "MEDIUM" or row[column] == "BAD":
            problematic.append(column)
    if problematic:
        dfPrintClusters = dfPrintClusters.append({"runNumber":runNumber, "problematicROCs":" ".join(problematic)}, ignore_index=True)
        #print(f"Run {runNumber}: Issue in clusters with", *problematic)

In [ ]:
from IPython.display import display, Markdown, Latex
display(Markdown(dfPrintClusters.to_markdown(index=False)))

In [ ]:

myPalette = np.array([920, 414, 801, 633],dtype=np.int32)
ROOT.gStyle.SetPalette(4,myPalette)
ROOT.gStyle.SetGridStyle(1)
qualityLabels = {"NULL":0,"GOOD":1,"MEDIUM":2,"BAD":3}

canvas = ROOT.TCanvas("QualityMatrixClusters","",1100,600)
canvas.SetLeftMargin(0.15)
canvas.SetBottomMargin(0.15)
canvas.SetRightMargin(0.15)
canvas.SetGrid()

qualityHistClusters = ROOT.TH2I("QualityMatrixClusters","Quality Matrix Clusters",
                        len(qualityDFClusters.index),min(qualityDFClusters.index),max(qualityDFClusters.index)+1,
                        len(qualityDFClusters.columns)-1,0,len(qualityDFClusters.columns)-1)
qualityHistClusters.SetCanExtend(ROOT.TH1.kAllAxes)
qualityHistClusters.SetStats(0)
for runIndex,run in enumerate(qualityDFClusters.runNumber):
    qualityHistClusters.GetXaxis().SetBinLabel(runIndex+1,str(run))
    qualityHistClusters.GetXaxis().SetTickLength( 0.03)
    for checkIndex,check in enumerate(qualityDFClusters.loc[:, qualityDFClusters.columns != "runNumber"].columns):
        #print(checkIndex,runIndex)
        qualityHistClusters.Fill(run,check,qualityLabels.get(qualityDFClusters.iloc[runIndex][check]))

qualityHistClusters.LabelsOption("u")
qualityHistClusters.Draw("COLZ")
qualityHistClusters.GetZaxis().SetRangeUser(-0.5,3.5);
qualityHistClusters.GetZaxis().SetTitle("Quality")
qualityHistClusters.LabelsOption("v")
canvas.Draw()

### Tracks observables

In [ ]:
qualityDF = pd.DataFrame({'runNumber':runList})
objects = ["hPhiAside;1","hPhiCside;1","hEta;1","hPt;1","hQOverPt;1","hSign;1",
           "hdEdxTotMIP_TPC;1","hNClustersAfterCuts;1","hdEdxVsTgl;1",
           "hdEdxTotMIPVsSnp_TPC;1"]
trending = "mean"
error = "meanError"
for objectName in objects:
    name = objectName[0:objectName.find(";")]
    [trend,canvas] = drawTrending(objectName,rootDataFile,names=runList,namesFromRunList=True,
                                   trend=trending,error=error,log="none",axis=1)
    [qualities,canvas] = checkTrending(trend,canvas=canvas,thresholds={"GOOD":1.5,"MEDIUM":3,"BAD":6})
    qualityDF[name] = qualities

In [ ]:
dfPrintTracks = pd.DataFrame(columns=["runNumber","Issues"])
for index, row in qualityDF.iterrows():
    runNumber = row["runNumber"]
    problematic = []
    for column in qualityDF.columns:
        if row[column] == "MEDIUM" or row[column] == "BAD":
            problematic.append(column)
    if problematic:
        dfPrintTracks = dfPrintTracks.append({"runNumber":runNumber, "Issues":" ".join(problematic)}, ignore_index=True)
        #print(f"Run {runNumber}: Issue in clusters with", *problematic)
display(Markdown(dfPrintTracks.to_markdown(index=False)))

In [ ]:
canvas = ROOT.TCanvas("QualityMatrixTracks","",1100,500)
canvas.SetLeftMargin(0.15)
canvas.SetBottomMargin(0.15)
canvas.SetRightMargin(0.15)
canvas.SetGrid()

qualityHist = ROOT.TH2I("QualityMatrixTracks","Quality Matrix Tracks Observables",
                        len(qualityDF.index),min(qualityDF.index),max(qualityDF.index)+1,
                        len(qualityDF.columns)-1,0,len(qualityDF.columns)-1)
qualityHist.SetCanExtend(ROOT.TH1.kAllAxes)
qualityHist.SetStats(0)
for runIndex,run in enumerate(qualityDF.runNumber):
    qualityHist.GetXaxis().SetBinLabel(runIndex+1,str(run))
    qualityHist.GetXaxis().SetTickLength( 0.03)
    for checkIndex,check in enumerate(qualityDF.loc[:, qualityDF.columns != "runNumber"].columns):
        #print(checkIndex,runIndex)
        qualityHist.Fill(run,check,qualityLabels.get(qualityDF.iloc[runIndex][check]))

qualityHist.LabelsOption("u")
qualityHist.Draw("COLZ")
qualityHist.GetZaxis().SetRangeUser(-0.5,3.5)
qualityHist.GetZaxis().SetTitle("Quality")
qualityHist.LabelsOption("v")

# Update the canvas
canvas.Update()
canvas.Draw()

---
## Comparing apass3 to apass2 with trends

In [ ]:
%jsroot on
objectName="hPt;1"
trending = "entries"
error = "stdDev"
leg = ROOT.TLegend()
[trend,canvas] = drawTrending(objectName,rootDataFile,names=runList,namesFromRunList=True,
                               trend=trending,error=error,yAxisRange=[0,750e9])
leg.AddEntry(trend, passName, "lep")
[trend2,canvas] = drawTrending(objectName,rootDataFile_apass1,names=runList,namesFromRunList=True,canvas=canvas,
                               trend=trending,error=error,histName="apass2")
leg.AddEntry(trend2, pass1Name, "lep")
leg.Draw()
trend.SetTitle("Number of tracks Trending")
canvas.Draw()

In [ ]:
%jsroot on
objectName="hPhiAside;1"
trending = "mean"
error = "meanError"
leg = ROOT.TLegend()
[trend,canvas] = drawTrending(objectName,rootDataFile,names=runList,namesFromRunList=True,
                               trend=trending,error=error,
                               log="none",yAxisRange=[3.12,3.18])
leg.AddEntry(trend, passName, "lep")
[trend2,canvas] = drawTrending(objectName,rootDataFile_apass1,names=runList,namesFromRunList=True,canvas=canvas,
                               trend=trending,error=error,
                               log="none",histName="apass2")
leg.AddEntry(trend2, pass1Name, "lep")
leg.Draw()
#trend.SetTitle("Number of tracks - Trend")
canvas.Draw()

In [ ]:
%jsroot on
objectName="hPhiCside;1"
trending = "mean"
error = "meanError"
leg = ROOT.TLegend()
[trend,canvas] = drawTrending(objectName,rootDataFile,names=runList,namesFromRunList=True,
                               trend=trending,error=error,
                               log="none",yAxisRange=[3.1,3.7])
leg.AddEntry(trend, passName, "lep")
[trend2,canvas] = drawTrending(objectName,rootDataFile_apass1,names=runList,namesFromRunList=True,canvas=canvas,
                               trend=trending,error=error,
                               log="none",histName="apass2")
leg.AddEntry(trend2, pass1Name, "lep")
leg.Draw()
#trend.SetTitle("Number of tracks - Trend")
canvas.Draw()

In [ ]:
%jsroot on
objectName="hEta;1"
trending = "mean"
error = "meanError"
leg = ROOT.TLegend()
[trend,canvas] = drawTrending(objectName,rootDataFile,names=runList,namesFromRunList=True,
                               trend=trending,error=error,
                               log="none",yAxisRange=[18e-3,60e-3])
leg.AddEntry(trend, passName, "lep")
[trend2,canvas] = drawTrending(objectName,rootDataFile_apass1,names=runList,namesFromRunList=True,canvas=canvas,
                               trend=trending,error=error,
                               log="none",histName="apass2")
leg.AddEntry(trend2, pass1Name, "lep")
leg.Draw()
#trend.SetTitle("Number of tracks - Trend")
canvas.Draw()

### pT trending
+ Change in pT tail for runs with high (500kHz) IR

In [ ]:
%jsroot on
objectName="hPt;1"
trending = "mean"
error = "meanError"
leg = ROOT.TLegend()
[trend,canvas] = drawTrending(objectName,rootDataFile,names=runList,namesFromRunList=True,
                               trend=trending,error=error,yAxisRange=[0.42,0.485],
                               log="none")
leg.AddEntry(trend, passName, "lep")
[trend2,canvas] = drawTrending(objectName,rootDataFile_apass1,names=runList,namesFromRunList=True,canvas=canvas,
                               trend=trending,error=error,yAxisRange=[0.42,0.465],
                               log="none",histName="apass2")
leg.AddEntry(trend2, pass1Name, "lep")
leg.Draw()
#trend.SetTitle("Number of tracks - Trend")
canvas.Draw()

In [ ]:
%jsroot on
objectName="hSign;1"
trending = "mean"
error = "meanError"
leg = ROOT.TLegend()
[trend,canvas] = drawTrending(objectName,rootDataFile,names=runList,namesFromRunList=True,
                               trend=trending,error=error,yAxisRange=[140E-3,180E-3],
                               log="none")
leg.AddEntry(trend, passName, "lep")
[trend2,canvas] = drawTrending(objectName,rootDataFile_apass1,names=runList,namesFromRunList=True,canvas=canvas,
                               trend=trending,error=error,yAxisRange=[150E-3,166E-3],
                               log="none",histName="apass2")
leg.AddEntry(trend2, pass1Name, "lep")
leg.Draw()
#trend.SetTitle("Number of tracks - Trend")
canvas.Draw()

### Time gain calibration

In [ ]:
%jsroot on
objectName="hdEdxTotMIP_TPC;1"
trending = "mean"
error = "meanError"
leg = ROOT.TLegend()
[trend,canvas] = drawTrending(objectName,rootDataFile,names=runList,namesFromRunList=True,
                               trend=trending,error=error,
                               log="none",yAxisRange=[52.5,55.5])
leg.AddEntry(trend, passName, "lep")
[trend2,canvas] = drawTrending(objectName,rootDataFile_apass1,names=runList,namesFromRunList=True,canvas=canvas,
                               trend=trending,error=error,
                               log="none",histName="apass2")
leg.AddEntry(trend2, pass1Name, "lep")
leg.Draw()
#trend.SetTitle("Number of tracks - Trend")
canvas.Draw()

In [ ]:
%jsroot on
objectName="hNClustersAfterCuts;1"
trending = "mean"
error = "meanError"
leg = ROOT.TLegend()
[trend,canvas] = drawTrending(objectName,rootDataFile,names=runList,namesFromRunList=True,
                               trend=trending,error=error,
                               log="none",yAxisRange=[90,118])
leg.AddEntry(trend, passName, "lep")
[trend2,canvas] = drawTrending(objectName,rootDataFile_apass1,names=runList,namesFromRunList=True,canvas=canvas,
                               trend=trending,error=error,
                               log="none",histName="apass2")
leg.AddEntry(trend2, pass1Name, "lep")
leg.Draw()
#trend.SetTitle("Number of tracks - Trend")
canvas.Draw()

In [ ]:
%jsroot on
objectName="hdEdxVsTgl;1"
trending = "mean"
error = "meanError"
leg = ROOT.TLegend()
[trend,canvas] = drawTrending(objectName,rootDataFile,names=runList,namesFromRunList=True,
                               trend=trending,error=error,axis=1,
                               log="none",yAxisRange=[10e-3,90e-3])
leg.AddEntry(trend, passName, "lep")
[trend2,canvas] = drawTrending(objectName,rootDataFile_apass1,names=runList,namesFromRunList=True,canvas=canvas,
                               trend=trending,error=error,axis=1,
                               log="none",histName="apass2")
leg.AddEntry(trend2, pass1Name, "lep")
leg.Draw()
trend.SetTitle("Track tan(lambda) - Trend")
canvas.Draw()

In [ ]:
%jsroot on
objectName="hdEdxTotMIPVsSnp_TPC;1"
trending = "mean"
error = "meanError"
leg = ROOT.TLegend()
[trend,canvas] = drawTrending(objectName,rootDataFile,names=runList,namesFromRunList=True,
                               trend=trending,error=error,axis=1,
                               log="none",yAxisRange=[-0.05,0.05],histName="apass2")
leg.AddEntry(trend, passName, "lep")
[trend2,canvas] = drawTrending(objectName,rootDataFile_apass1,names=runList,namesFromRunList=True,canvas=canvas,
                               trend=trending,error=error,axis=1,
                               log="none",histName="apass3")
leg.AddEntry(trend2, pass1Name, "lep")
leg.Draw()
trend.SetTitle("Track sin(phi) - Trend")
canvas.Draw()

---
## apass3 only trends with checks

In [ ]:
thresholds = {"GOOD":3,"MEDIUM":6,"BAD":9}

In [ ]:
%jsroot on
objectName="hPhiAside;1"
trending = "mean"
error = "meanError"
leg = ROOT.TLegend()
[trend,canvas] = drawTrending(objectName,rootDataFile,names=runList,namesFromRunList=True,
                               trend=trending,error=error,
                               log="none",yAxisRange=[2.12,4.18])
[qualities,canvas] = checkTrending(trend,canvas=canvas,thresholds=thresholds)
#qualityDF[objectName] = qualities
canvas.Draw()

In [ ]:
%jsroot on
objectName="hPhiCside;1"
trending = "mean"
error = "meanError"
leg = ROOT.TLegend()
[trend,canvas] = drawTrending(objectName,rootDataFile,names=runList,namesFromRunList=True,
                               trend=trending,error=error,
                               log="none",yAxisRange=[3.12,3.18])
[qualities,canvas] = checkTrending(trend,canvas=canvas,thresholds=thresholds)
#qualityDF[objectName] = qualities
#trend.SetTitle("Number of tracks - Trend")
canvas.Draw()

In [ ]:
%jsroot on
objectName="hEta;1"
trending = "mean"
error = "meanError"
leg = ROOT.TLegend()
[trend,canvas] = drawTrending(objectName,rootDataFile,names=runList,namesFromRunList=True,
                               trend=trending,error=error,
                               log="none",yAxisRange=[18e-3,60e-3])
[qualities,canvas] = checkTrending(trend,canvas=canvas,thresholds={"GOOD":3,"MEDIUM":6,"BAD":9})
#qualityDF[objectName] = qualities
#trend.SetTitle("Number of tracks - Trend")
canvas.Draw()

In [ ]:
%jsroot on
objectName="hPt;1"
trending = "mean"
error = "meanError"
leg = ROOT.TLegend()
[trend,canvas] = drawTrending(objectName,rootDataFile,names=runList,namesFromRunList=True,
                               trend=trending,error=error,
                               log="none",yAxisRange=[18e-3,60e-3])
[qualities,canvas] = checkTrending(trend,canvas=canvas,thresholds={"GOOD":3,"MEDIUM":6,"BAD":9})
#qualityDF[objectName] = qualities
#trend.SetTitle("Number of tracks - Trend")
canvas.Draw()

In [ ]:
%jsroot on
objectName="hSign;1"
trending = "mean"
error = "meanError"
leg = ROOT.TLegend()
[trend,canvas] = drawTrending(objectName,rootDataFile,names=runList,namesFromRunList=True,
                               trend=trending,error=error,
                               log="none",yAxisRange=[18e-3,60e-3])
[qualities,canvas] = checkTrending(trend,canvas=canvas,thresholds={"GOOD":3,"MEDIUM":6,"BAD":9})
#qualityDF[objectName] = qualities
#trend.SetTitle("Number of tracks - Trend")
canvas.Draw()

In [ ]:
%jsroot on
objectName="hdEdxTotMIP_TPC;1"
trending = "mean"
error = "meanError"
leg = ROOT.TLegend()
[trend,canvas] = drawTrending(objectName,rootDataFile,names=runList,namesFromRunList=True,
                               trend=trending,error=error,
                               log="none",yAxisRange=[18e-3,60e-3])
[qualities,canvas] = checkTrending(trend,canvas=canvas,thresholds={"GOOD":3,"MEDIUM":6,"BAD":9})
#qualityDF[objectName] = qualities
#trend.SetTitle("Number of tracks - Trend")
canvas.Draw()

In [ ]:
%jsroot on
objectName="hNClustersAfterCuts;1"
trending = "mean"
error = "meanError"
leg = ROOT.TLegend()
[trend,canvas] = drawTrending(objectName,rootDataFile,names=runList,namesFromRunList=True,
                               trend=trending,error=error,
                               log="none")
[qualities,canvas] = checkTrending(trend,canvas=canvas,thresholds={"GOOD":3,"MEDIUM":6,"BAD":9})
#qualityDF[objectName] = qualities
#trend.SetTitle("Number of tracks - Trend")
canvas.Draw()

In [ ]:
%jsroot on
objectName="hdEdxVsTgl;1"
trending = "mean"
error = "meanError"
leg = ROOT.TLegend()
[trend,canvas] = drawTrending(objectName,rootDataFile,names=runList,namesFromRunList=True,
                               trend=trending,error=error,
                               log="none")
[qualities,canvas] = checkTrending(trend,canvas=canvas,thresholds={"GOOD":3,"MEDIUM":6,"BAD":9})
#qualityDF[objectName] = qualities
#trend.SetTitle("Number of tracks - Trend")
canvas.Draw()

In [ ]:
%jsroot on
objectName="hdEdxTotMIPVsSnp_TPC;1"
trending = "mean"
error = "meanError"
leg = ROOT.TLegend()
[trend,canvas] = drawTrending(objectName,rootDataFile,names=runList,namesFromRunList=True,
                               trend=trending,error=error,axis=1,
                               log="none")
[qualities,canvas] = checkTrending(trend,canvas=canvas,thresholds={"GOOD":3,"MEDIUM":6,"BAD":9})
#qualityDF[objectName] = qualities
#trend.SetTitle("Number of tracks - Trend")
canvas.Draw()

---
## Plots from apass3

In [ ]:
ROOT.gStyle.SetPalette(57)

## Clusters QC
+ Cluster maps look OK
Example run with problematic C side ROCs (`528602`) shown

In [ ]:
%jsroot on
objectName = "c_N_Clusters"
c = rootDataFile[2].ClusterQC.Get(objectName)
c.Draw()

In [ ]:
%jsroot on
objectName = "c_Q_Tot"
c = rootDataFile[4].ClusterQC.Get(objectName)
c.Draw()

---
## Tracks QC
### Geometrical distributions of tracks
**Phi distribution A-side tracks:**
+ Run `529310` problems with sector A4

In [ ]:
%jsroot on
objectName="hPhiAside"
[hist,legend,canvas,pad1,histComp] = drawHistograms(objectName,rootDataFile,normalize=True,legend=False,legendNames=runList,pads=True,
                                           compareTo=rootDataFile_apass1,drawOption="SAME HIST PLC PMC",yAxisRange=[0,4e-3],
                                           check=qualityDF[objectName])

leg = ROOT.TLegend()
leg.AddEntry(hist, passName, "lep")
leg.AddEntry(histComp, pass1Name, "lep")
#Sector boundatries
for i in range(len(rootDataFile)):
    pad1.cd(i+1)
    
    sectorBoundary = ROOT.TLine(0,hist.GetMinimum(),0,hist.GetMaximum())
    sectorBoundary.SetLineStyle(2)
    sectorNum = ROOT.TText(.5,.5,"test")
    sectorNum.SetTextAlign(22)
    sectorNum.SetTextColor(ROOT.kGray+1)
    sectorNum.SetTextFont(0)
    for j in range(19):
        sectorBoundary.DrawLine((j*2*math.pi)/18,hist.GetMinimum(),(j*2*math.pi)/18,hist.GetMaximum())
        if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,hist.GetMaximum(),str(j))
    leg.Draw()

canvas.Draw()

**Phi distribution C-side tracks:**
+ Run `529338` problems with sector C12

In [ ]:
%jsroot on
objectName="hPhiCside"
[hist,legend,canvas,pad1,histComp] = drawHistograms(objectName,rootDataFile,normalize=True,legend=False,legendNames=runList,pads=True,
                                           compareTo=rootDataFile_apass1,drawOption="SAME HIST PLC PMC",yAxisRange=[0,4e-3],
                                                   check=qualityDF[objectName])
leg = ROOT.TLegend()
leg.AddEntry(hist, passName, "lep")
leg.AddEntry(histComp, pass1Name, "lep")
#Sector boundatries
for i in range(len(rootDataFile)):
    pad1.cd(i+1)
    sectorBoundary = ROOT.TLine(0,hist.GetMinimum(),0,hist.GetMaximum())
    sectorBoundary.SetLineStyle(2)
    sectorNum = ROOT.TText(.5,.5,"test")
    sectorNum.SetTextAlign(22)
    sectorNum.SetTextColor(ROOT.kGray+1)
    sectorNum.SetTextFont(0)
    for j in range(19):
        sectorBoundary.DrawLine((j*2*math.pi)/18,hist.GetMinimum(),(j*2*math.pi)/18,hist.GetMaximum())
        if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,hist.GetMaximum(),str(j))
    leg.Draw()
canvas.Draw()

### Eta distribution

In [ ]:
%jsroot on
objectName="hPhiCside"
[hist,legend,canvas,pad1,histComp] = drawHistograms(objectName,rootDataFile[8:9],normalize=True,legend=False,legendNames=runList[8:9],pads=True,
                                           compareTo=rootDataFile_apass1[8:9],drawOption="SAME HIST PLC PMC",yAxisRange=[2e-3,3.5e-3])
leg = ROOT.TLegend()
leg.AddEntry(hist, passName, "lep")
leg.AddEntry(histComp, pass1Name, "lep")
pad1.cd(1)
sectorBoundary = ROOT.TLine(0,hist.GetMinimum(),0,hist.GetMaximum())
sectorBoundary.SetLineStyle(2)
sectorNum = ROOT.TText(.5,.5,"test")
sectorNum.SetTextAlign(22)
sectorNum.SetTextColor(ROOT.kGray+1)
sectorNum.SetTextFont(0)
for j in range(19):
    sectorBoundary.DrawLine((j*2*math.pi)/18,hist.GetMinimum(),(j*2*math.pi)/18,hist.GetMaximum())
    if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,hist.GetMaximum(),str(j))
leg.Draw()
canvas.Draw()

In [ ]:
%jsroot on
objectName="hEta"
[hist,legend,canvas,pad1,histComp] = drawHistograms(objectName,rootDataFile[8:9],normalize=True,legend=False,legendNames=runList[8:9],pads=True,
                                           compareTo=rootDataFile_apass1[8:9],drawOption="SAME HIST PLC PMC",yAxisRange=[2e-3,9e-3])
leg = ROOT.TLegend()
leg.AddEntry(hist, passName, "lep")
leg.AddEntry(histComp, pass1Name, "lep")
leg.Draw()
canvas.Draw()

In [ ]:
%jsroot on
objectName="hPt"
[hist,legend,canvas,pad1,histComp] = drawHistograms(objectName,rootDataFile[8:9],normalize=True,legend=False,legendNames=runList[8:9],pads=True,
                                           compareTo=rootDataFile_apass1[8:9],drawOption="SAME HIST PLC PMC")
leg = ROOT.TLegend()
leg.AddEntry(hist, passName, "lep")
leg.AddEntry(histComp, pass1Name, "lep")
leg.Draw()
canvas.Draw()

In [ ]:
%jsroot on
objectName="h2DEtaPhi"
[hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,normalize=False,legend=False,yAxisRange = [-1.1,1.1],legendNames=runList,pads=True,drawOption="COLZ")
canvas.Draw()

### Track properties
**Transverse momentum**

In [ ]:
objectName="hPt"
[hist,legend,canvas,pad1,histComp] = drawHistograms(objectName,rootDataFile,normalize=True,legend=False,legendNames=runList,pads=True,
                                           compareTo=rootDataFile_apass1,drawOption="SAME HIST PLC PMC",log="logxy",check=qualityDF[objectName])
leg = ROOT.TLegend()
leg.AddEntry(hist, passName, "lep")
leg.AddEntry(histComp, pass1Name, "lep")
for i in range(len(rootDataFile)):
    pad1.cd(i+1)
    leg.Draw()
canvas.Draw()

**Track sign**
+ Ratio as expected

In [ ]:
objectName="hSign"
[hist,legend,canvas,pad1,histComp] = drawHistograms(objectName,rootDataFile,normalize=True,legend=False,legendNames=runList,pads=True,
                                           compareTo=rootDataFile_apass1,drawOption="SAME HIST PLC PMC",check=qualityDF[objectName])
leg = ROOT.TLegend()
leg.AddEntry(hist, passName, "lep")
leg.AddEntry(histComp, pass1Name, "lep")
for i in range(len(rootDataFile)):
    pad1.cd(i+1)
    leg.Draw()
canvas.Draw()

**Charge / transverse momentum**
+ All runs OK

In [ ]:
%jsroot on
objectName="hQOverPt"
[hist,legend,canvas,pad1,histComp] = drawHistograms(objectName,rootDataFile,normalize=True,legend=False,legendNames=runList,pads=True,
                                           compareTo=rootDataFile_apass1,drawOption="SAME HIST PLC PMC",check=qualityDF[objectName])
leg = ROOT.TLegend()
leg.AddEntry(hist, passName, "lep")
leg.AddEntry(histComp, pass1Name, "lep")
for i in range(len(rootDataFile)):
    pad1.cd(i+1)
    leg.Draw()
canvas.Draw()

### Number of clusters per track
+ All OK

In [ ]:
%jsroot on
objectName="hNClustersAfterCuts"
[hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,normalize=True,legend=True,xAxisRange = [50,160],yAxisRange=[0,30e-3],legendNames=runList,pads=False)
canvas.Draw()

In [ ]:
%jsroot on
objectName="hNClustersAfterCuts"
[hist,legend,canvas,pad1,histComp] = drawHistograms(objectName,rootDataFile,normalize=True,legend=False,legendNames=runList,pads=True,
                                           compareTo=rootDataFile_apass1,drawOption="SAME HIST PLC PMC",xAxisRange = [50,160],yAxisRange=[0,2E-2],check=qualityDF[objectName])
leg = ROOT.TLegend()
leg.AddEntry(hist, passName, "lep")
leg.AddEntry(histComp, pass1Name, "lep")
for i in range(len(rootDataFile)):
    pad1.cd(i+1)
    leg.Draw()
canvas.Draw()

### nClusters vs phi

In [ ]:
%jsroot on
objectName="h2DNClustersPhiAside"
[hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,normalize=False,legend=False,yAxisRange = [50,170],legendNames=runList,pads=True,drawOption="COLZ")
canvas.Draw()

In [ ]:
%jsroot on
objectName="h2DNClustersPhiCside"
[hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,normalize=False,legend=False,yAxisRange = [50,170],legendNames=runList,pads=True,drawOption="COLZ")
canvas.Draw()

### nClusters vs eta
+ All runs OK

In [ ]:
%jsroot on
objectName="h2DNClustersEta"
[hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,normalize=False,legend=False,xAxisRange = [-1.1,1.1],yAxisRange = [50,170],legendNames=runList,pads=True,drawOption="COLZ")
canvas.Draw()

### nClusters vs transverse momentum
+ All runs OK

In [ ]:
%jsroot on
objectName="h2DNClustersPt"
[hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,normalize=False,legend=False,log="logx",yAxisRange = [50,170],legendNames=runList,pads=True,drawOption="COLZ")
canvas.Draw()

### PID
+ All runs OK

In [ ]:
%jsroot on
objectName="hdEdxTotMIP_TPC"
#quality = checkHistograms(objectName,rootDataFile,check="(histogram.Fit(\"gaus\",\"Sq\",\"\",40,60)) and 49<histogram.GetFunction(\"gaus\").GetParameter(1)<51")
[hist,legend,canvas,pad1,histComp] = drawHistograms(objectName,rootDataFile,normalize=True,legend=False,legendNames=runList,pads=True,log="logx",
                                           compareTo=rootDataFile_apass1,drawOption="SAME HIST PLC PMC",yAxisRange=[0,0.12],check=qualityDF[objectName])

leg = ROOT.TLegend()
leg.AddEntry(hist, passName, "lep")
leg.AddEntry(histComp, pass1Name, "lep")
for i in range(len(rootDataFile)):
    pad1.cd(i+1)
    sectorBoundary = ROOT.TLine()
    sectorBoundary.SetLineStyle(1)
    sectorBoundary.SetLineColor(ROOT.kGreen-2)
    sectorBoundary.SetLineWidth(2)
    sectorBoundary.DrawLine(50,hist.GetMinimum(),50,0.12)
    leg.Draw()
    
canvas.Draw()

## TPC Gain calibration
**dEdx of MIP vs sector**
+ All runs have uniform dEdx response in sector

In [ ]:
%jsroot on
objectName="hdEdxTotMIPVsSec_TPC"
[hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,normalize=False,legend=False,log="logy",legendNames=runList,pads=True,drawOption="COLZ")
for i in range(len(rootDataFile)):
    pad1.cd(i+1)
    sectorBoundary = ROOT.TLine()
    sectorBoundary.SetLineStyle(2)
    sectorBoundary.SetLineColor(ROOT.kBlack)
    sectorBoundary.SetLineWidth(2)
    sectorBoundary.DrawLine(0,50,36,50)

canvas.Draw()

**dEdx of Mip vs nClusters**
* All runs OK

In [ ]:
%jsroot on
objectName="hdEdxTotMIPVsNcl_TPC"
[hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,normalize=False,legend=False,log="logz",xAxisRange = [55,180],legendNames=runList,pads=True,drawOption="COLZ")
for i in range(len(rootDataFile)):
    pad1.cd(i+1)
    sectorBoundary = ROOT.TLine()
    sectorBoundary.SetLineStyle(2)
    sectorBoundary.SetLineColor(ROOT.kBlack)
    sectorBoundary.SetLineWidth(2)
    sectorBoundary.DrawLine(55,50,180,50)
canvas.Draw()

**dEdx of Mip vs sin(phi)**
* All runs OK

In [ ]:
%jsroot on
objectName="hdEdxTotMIPVsSnp_TPC"
[hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,normalize=False,legend=False,log="logz",legendNames=runList,pads=True,drawOption="COLZ")
for i in range(len(rootDataFile)):
    pad1.cd(i+1)
    sectorBoundary = ROOT.TLine()
    sectorBoundary.SetLineStyle(2)
    sectorBoundary.SetLineColor(ROOT.kBlack)
    sectorBoundary.SetLineWidth(2)
    sectorBoundary.DrawLine(hist.GetXaxis().GetXmin(),50,hist.GetXaxis().GetXmax(),50)
canvas.Draw()

**dEdx of Mip vs tan(lambda)**
* All runs OK

In [ ]:
%jsroot on
objectName="hdEdxTotMIPVsTgl_TPC"
[hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,normalize=False,legend=False,log="logz",legendNames=runList,pads=True,drawOption="COLZ")
for i in range(len(rootDataFile)):
    pad1.cd(i+1)
    sectorBoundary = ROOT.TLine()
    sectorBoundary.SetLineStyle(2)
    sectorBoundary.SetLineColor(ROOT.kBlack)
    sectorBoundary.SetLineWidth(2)
    sectorBoundary.DrawLine(hist.GetXaxis().GetXmin(),50,hist.GetXaxis().GetXmax(),50)
canvas.Draw()

### dEdx vs p
+ All runs OK

In [ ]:
%jsroot on
from TPCQCVis.src.drawBetheBloch import *
objectName="hdEdxTotVsP_TPC"
[hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,normalize=False,legend=False,log="logxyz",legendNames=runList,pads=True,drawOption="COLZ")
betheBlochLines = getBetheBloch(runList)
pad1 = drawBetheBloch(betheBlochLines,pad1)
canvas.Draw()